
# How much of the web runs on renewable power?

I'm trying to work this out as some research for the Planet Friendly Web Guide, as I can't find much info online myself.

Here's my current thinking on how you might do this. There are obvious holes in this, but it's a start, and having _something_ publicly accessible presumably better than what we have now, this is _nowt_.


## My current thinking about how to do this

- Take a dataset that is representative of the entire web, listing the most popular sites
- Find a away to tell if these sites are running on renewable power
- Run this check on the sites
- Share results

I'll explain these in more detail.


### Find a dataset that's representative

Right now, the de-factor dataset used to represent how the web is used, at least for web performance optimisation and marketing is Alexa's list of the top million sites, as described on [the HTTP Archive site][1].

[1]: https://httparchive.org/about.php#faq

Iniitally I thought you might need to come up with some on Big Query er, query, like described here on [Ilya Grigorik's website](https://www.igvita.com/2013/06/20/http-archive-bigquery-web-performance-answers/). This saves needing to run our own infra, and should make reproducible.

But given we only need the url, and rank for now we might be able to do this without needing to learn big query, or how to make API calls when running a query on the BigQuery.

The list of the top million is downloadable as a compressed file, that's about 9mb compressed, and 23mb uncompressed. This is downloadable from an Amazon S3 bucket, at the link below

http://s3.amazonaws.com/alexa-static/top-1m.csv.zip

### Find a way to tell if it's running on renewable power

This in itself is a hellishly complex topic - as there loads of ways to declare that your infrastructure is running on renewable power, from _actually using renewables directly_ to using various financial instruments to buy the same about renewable power you're really using in a non-renewably powered datacentre, to another form of offsets.

But for now, to start with, let's try using the [Green Web API](https://www.thegreenwebfoundation.org/green-web-feed/) - the API is simple to understand, and in use.

This API checks a domain against who is hosting it, and checks if the hosting company has declared they're running on renewables. It's not clear to me yet what specific checks there, and _how_ they do this, but again, it's our first pass at this.

### Run this check

It's not technically complex to loop through a csv file, and for each row hit an API. In python the pseudocode might look like this:

```python

green_or_not_list = []

for row in open('path/to/dataset.csv'):
    # do some stuff to find the right column, or tell python we're 
    # looking at a CSV file
    site_url = pull_url_column_from(row)
    
    result = check_against_green_web_api(site_url)
    green_or_not_list.push(result)

```

Thing is this, this involves hitting the API a million times if there are a million rows in the dataset.

It's in the very least polite to contact the nice folks at Green Web before we hit their API a million times - one for each url in the Alexa dataset though.

The nice thing in the dataset from Alex is that the csv file as two columns, domain and rank, we can do this with the first 10, or maybe 100 results, and get something halfway interesting back, without hosing somebody else's API.

### Share Results

We should really to make this reproducible.

It looks like packaging this up as a [binder][] would be a good step for this. I haven't made one before, but it's worth a try, right?

[binder]: https://mybinder.org/

------------------------------------------


In [1]:
import requests
import zipfile

req = requests.get('http://s3.amazonaws.com/alexa-static/top-1m.csv.zip', stream=True)

with open('top_1m_sites.csv.zip', 'wb') as fd:
    for chunk in req.iter_content(chunk_size=128):
        fd.write(chunk)

req

<Response [200]>

We have a zip file, so let's unzip it to see what's inside - the file should be called `top-1m.csv`

In [2]:
zip_ref = zipfile.ZipFile('top_1m_sites.csv.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

A file with 1 million rows isn't big, but it's not small either, and let's try using pandas to cut it down quickly

In [3]:
import pandas as pd

top_1m_sites = pd.read_csv('top-1m.csv', header=None, names=['rank', 'domain'])

top_10_sites = top_1m_sites.head(10)

top_10_sites


,rank,domain
0,1,youtube.com
1,2,google.com
2,3,facebook.com
3,4,baidu.com
4,5,wikipedia.org
5,6,reddit.com
6,7,yahoo.com
7,8,qq.com
8,9,taobao.com
9,10,sohu.com


Okay, now we have a list, we can add our column to check against the Green Web project, to see if it's running on renewable power or not

In [4]:
import requests


def check_for_green_power(domain):
    
    api_url = "http://api.thegreenwebfoundation.org/greencheck/{}".format(domain)
    res = requests.get(api_url)
    json_resp = res.json()
    
    # lets keep it simple for now
    return json_resp['green']

So we now have our dataframe listing all the sites, and a function to check if the domain is running on green power.

The final step is to define a third column based on the results of calling `check_for_green_power` on the domain in this row.

Sigh… I've _totally_ forgot how to do this. Let's park it for now, as we need to probably be considerate with thei green web API, and add something at our end to rate limit it.


### After asking around

Ah, after a few false starts here's how to do it. As far as I can tell, you need to do this in two separate steps, otherwise the dataframe starts whining about copying with a message along the lines of:


```
SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
```

In [5]:
green_or_not = top_10_sites["domain"].apply(check_for_green_power)

green_or_not


KeyError: 'green'

Now we have our set of results for the list of domains we can then assign it to the dataframe.

In [ ]:
top_10_sites.assign(green_or_not=green_or_not)

In [ ]:
top_10_sites.to_csv('top-10-sites-with-greencheck.csv')

### So, how much of the internet is green?

If we take the top 10, we see 4 sites listed as running on renewable power (and three of them are google), and 6 running on fossil fuels.


# Update, with new data - exciting!

So, after getting in touch with Rene at the Greenweb Foundation, I've built a screen scraper to check rather more than 10, so we have a more meaningful dataset.

In fact, we're taking the top 1 million domains from Alexa, then checking the top 100k against the this Greenweb API. 

Because things break on the internet, and running a job like this takes a long time, and some requests might fail, I used the scrapy framework, and wrote a scraper using it, so one failed request doesn't mean I need to start a job from scratch.

You can see the [code for the scraper on github](https://github.com/productscience/pfw-top-mil) (pull requests gladly accepted).

Anyway, we now have a dataset of which of the [top 100k domains run on green power, and who is hosting them](https://datbase.org/view?query=8be6d29e63b010bb4240dc366e578e1f7f91e89a73062a7802c5b83d9cc793fe).

Let's see if we can find anything interesting...


In [7]:
top_100k_domains = pd.read_csv('top-100k-domains-green-or-not.csv')

In [8]:
top_100k_domains.head(5)

,_type,data,error,green,hostedby,hostedbyid,hostedbywebsite,icon,iconurl,partner,url
0,dict,True,NaN,True,Google Inc.,595.0,www.google.com,NaN,NaN,NaN,youtube.com
1,dict,True,NaN,True,Google Inc.,595.0,www.google.com,NaN,NaN,NaN,google.com
2,dict,True,NaN,True,Facebook,475.0,www.facebook.com,NaN,NaN,NaN,facebook.com
3,dict,True,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,baidu.com
4,dict,True,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,wikipedia.org


Okay, there's something interesting here. I'm not too handy with Pandas, but there's something of interest, and I'm going to cheat, but adding a screenshot from some exploration in OpenRefine:

![openrefine-screenshot-of-top100k-analysis](./openrefine-screenshot-of-top100k-analysis.png)


Okay, this is interesting. According to this, Hetzner is the biggest host of green domains, even bigger than Google. Who knew?

I'm not sure where Amazon is on this list, and the criteria we use for deciding whether something is green is something we'd probably need to clarify, but hey - this is a start, right?

### Update  with help from friends

So, one very helpful [MaikRos](https://github.com/MaikRos) showed me what I needed to do this check using just Pandas by itself. Thanks amigo!

I find the image above useful, but sharing the code below makes it easier to reproduce the claims listed.

In [12]:
top_100k_domains.groupby('hostedby').count()['green'].sort_values(ascending=False).head()

hostedby
Hetzner Online AG    2358
Google Inc.          1942
LeaseWeb              465
Serverius             315
1&1                   280
Name: green, dtype: int64